# 🚀 Showcase: 2D to 360° Video Generation (Final, Bulletproof Version)

This notebook provides a comprehensive and robust showcase for the 'Beyond the Frame' project. It has been completely redesigned to be as reliable as possible, with verification at every step.

### **Instructions**

1.  **Read Each Step Carefully:** Follow the instructions in the markdown cells. Do not skip steps.
2.  **GPU Runtime:** Ensure your runtime is set to use a GPU (`Runtime` > `Change runtime type` > `T4 GPU`).
3.  **Run Cells in Order:** Execute each cell from top to bottom.

### **Step 0: Factory Reset (Optional but Recommended)**

If you have run this notebook before, running the cell below will delete the old `argus-code` directory. This prevents pathing conflicts and ensures a clean setup.

In [ ]:
print('Cleaning up previous runs...')
!rm -rf argus-code
print('✅ Cleanup complete.')

### **Step 1: Install Conda & Restart Kernel**

⚠️ **CRITICAL STEP** ⚠️

Running the cell below will install `conda` and **automatically restart the Colab kernel.** This is expected and necessary.

**After the kernel restarts, you must continue running the cells from Step 2.**

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

--- 
### **Step 2: Full Setup & Verification (Run This After Kernel Restart)**

**IMPORTANT:** Run this single cell *after* the kernel has restarted from Step 1. 

This cell will perform the entire setup and **verify every single step.** If any step fails, it will print a `FATAL` error and stop execution.

In [ ]:
import os
import sys
import subprocess

def run_and_verify(command, check_path=None, failure_message=''):
    print(f'--- Running: {failure_message} ---')
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f'❌ FATAL: {failure_message} failed.')
        print(f'--- STDOUT ---\n{result.stdout}')
        print(f'--- STDERR ---\n{result.stderr}')
        sys.exit(1)
    if check_path and not os.path.exists(check_path):
        print(f'❌ FATAL: Verification failed. Expected path does not exist: {check_path}')
        sys.exit(1)
    print(f'✅ {failure_message} successful.')

# --- Step 2.1: Clone Repository ---
run_and_verify('git clone --recurse-submodules https://github.com/Red-Fairy/argus-code', 
                 check_path='argus-code', 
                 failure_message='Cloning repository')
os.chdir('argus-code')

# --- Step 2.2: Install Dependencies ---
run_and_verify('conda install -y -c pytorch -c nvidia faiss-gpu=1.8.0 cython', 
                 failure_message='Conda installation')
run_and_verify('pip install -q -r requirements.txt', 
                 failure_message='Pip requirements installation')
run_and_verify('pip install -q yt-dlp', 
                 failure_message='Pip yt-dlp installation')

# --- Step 2.3: Compile ASMK ---
run_and_verify('git clone https://github.com/jenicek/asmk', 
                 check_path='asmk', 
                 failure_message='Cloning ASMK')
os.chdir('asmk/cython/')
run_and_verify('cythonize *.pyx', 
                 failure_message='Cythonizing ASMK')
os.chdir('..')
run_and_verify('pip install .', 
                 failure_message='Installing ASMK')
os.chdir('..')

# --- Step 2.4: Download Models ---
os.makedirs('checkpoints', exist_ok=True)
run_and_verify('wget -q https://files.catbox.moe/t53m9y.zip -O checkpoints/pretrained-weights.zip', 
                 check_path='checkpoints/pretrained-weights.zip', 
                 failure_message='Downloading main model')

os.makedirs('mega-sam/Depth-Anything/checkpoints', exist_ok=True)
run_and_verify('wget -q https://huggingface.co/spaces/LiheYoung/Depth-Anything/resolve/main/checkpoints/depth_anything_vitl14.pth -O mega-sam/Depth-Anything/checkpoints/depth_anything_vitl14.pth', 
                 check_path='mega-sam/Depth-Anything/checkpoints/depth_anything_vitl14.pth', 
                 failure_message='Downloading DepthAnything model')

os.makedirs('mega-sam/cvd_opt', exist_ok=True)
run_and_verify('wget -q https://huggingface.co/ddrfan/RAFT/resolve/bfb9a87633743408b9c13c20b9ed43dbe5b83617/raft-things.pth -O mega-sam/cvd_opt/raft-things.pth', 
                 check_path='mega-sam/cvd_opt/raft-things.pth', 
                 failure_message='Downloading RAFT model')

# --- Step 2.5: Unzip and Verify Final Files ---
run_and_verify('unzip -q checkpoints/pretrained-weights.zip -d checkpoints/', 
                 failure_message='Unzipping main model')

files_to_check = [
    'checkpoints/pretrained-weights/unet/diffusion_pytorch_model.safetensors',
    'mega-sam/Depth-Anything/checkpoints/depth_anything_vitl14.pth',
    'mega-sam/cvd_opt/raft-things.pth'
]
for f in files_to_check:
    run_and_verify('', check_path=f, failure_message=f'Verifying final file: {f}')

print('\n--- ✅✅✅ SETUP COMPLETE AND FULLY VERIFIED ✅✅✅ ---')

### **Step 3: Choose Your Demo**

**Important:** Only run the cells for the option you want to use.

---

#### Option 1: Interactive Demo (Gradio Web UI)

Run the cell below to launch a Gradio web interface. A public URL will be generated. Click it to open the demo in a new tab, where you can upload your own video for processing.

In [ ]:
!python gradio_demo.py --share

---

#### Option 2: Automated Processing from a URL

Run the two cells below to automatically download a video from a URL and process it using the command-line script.

In [ ]:
# --- You can change this URL to any video you want to process ---
video_url = 'https://www.youtube.com/watch?v=YE7VzlLtp-4' # Big Buck Bunny (placeholder)

!mkdir -p input_videos
!yt-dlp -o 'input_videos/video.mp4' -f 'best[height<=720]' {video_url}

video_path = 'input_videos/video.mp4'
if os.path.exists(video_path):
    print(f'✅ Video downloaded successfully to: {video_path}')
else:
    print(f'❌ FAILED to download video. The URL may be invalid.')

In [ ]:
if os.path.exists(video_path):
    print('Starting automated processing...')
    unet_path = './checkpoints/pretrained-weights'
    save_folder = './outputs'
    
    !python inference.py \
        --unet_path {unet_path} \
        --video_path {video_path} \
        --val_save_folder {save_folder} \
        --guidance_scale 3 \
        --num_inference_steps 25
    
    print(f'\n--- Processing Complete! ---\n')
    print(f'Check the `{save_folder}` directory for your generated 360° video.')
else:
    print('Skipping processing because video download failed.')